# BCS Data Analysis Project

To frame our analysis, we must start with a concrete question to ground ourselves.
In this case, we're asking ourselves; So what's up with counter heals?
* how many do decks play on average?
* how do high performing decks relate to this average?
* how about low performing decks?

In order to answer this we'll need to:
* obtain HTML page for each decklog
* add each deck's info. into a data base (each card name and it's quantity)
* determine the names we want to know about
* query and graph the data

# Step 1 + 2
After a bit of wrangling, I've reworked these steps to start by scraping information about each event of the announcement page, so we can have information about the date of each event, and a second bit of scraping to get the link extension for each of those events off the event report page. 

Bushiroad might be weird with how their links are patterened or changed, as well as not maintaining event order between site pages, so it is wise to manually inspect the results of the scraping before feeding the results into our pipeline for data collection.

For more information about the scraping, and the schema of the table below, check out the Data Collection and Data Wrangling sections

In [ ]:
# BCS 25/26 Event locations
# These must manually be extracted from the bushiroad site, 
# https://en.cf-vanguard.com/event/bcs2526/
BASE_URL = "https://en.cf-vanguard.com/event/bcs2526/"

final_event_list=\
[
    ## ['illinois', 'Rosemont', 'NA', 'October 4, 2025'],
    ## ['mexico', 'Mexico', 'NA', 'November 8, 2025'],
    ## ['bcs2526-houston-tx', 'TX', 'NA', 'November 22, 2025'],
    ## ['bcs2526-california', 'LA', 'NA', 'December 6, 2025'],
    ## ['vancouver', 'BC', 'NA', 'January 10, 2026'],
    ## ['argentina', 'Argentina', 'NA', 'January 17, 2026'],
    ## ['duluth', 'Duluth', 'NA', 'January 17, 2026'],
    ## ['puerto-rico', 'Puerto Rico', 'NA', 'January 24, 2026'],
    
    ## The link will need testing
    # ['toronto-ca', 'Toronto', 'NA', 'February 14, 2026'],
    # ['san-jose-cr', 'Costa Rica', 'NA', 'February 21, 2026'],
    # ['philadelphia-pa', 'Philadelphia', 'NA', 'March 21, 2026'],

    
    ## ['modling-austria', 'Austria', 'EU', 'November 15, 2025'],
    ## ['italy', 'Italy', 'EU', 'December 13, 2025'],
    ## ['spain', 'Spain', 'EU', 'January 3, 2026'],
    ## ['france', 'France', 'EU', 'February 7, 2026'],
    
    ## These links will need testing
    # ['bochum', 'Germany', 'EU', 'February 21, 2026'],
    # ['greece', 'Greece', 'EU', 'March 7, 2026'],
    # ['manchester', 'United Kingdom', 'EU', 'March 21, 2026'],

    
    ## ['ho-chi-minh-city-vietnam', 'Vietnam', 'AO', 'November 2, 2025'],
    ## ['surabaya-indonesia', 'Indonesia', 'AO', 'November 16, 2025'],
    ## ['bcs2526-malaysia', 'Malaysia', 'AO', 'December 6, 2025'],
    ## ['manila', 'Philippines', 'AO', 'January 17, 2026'],
    ## ['singapore', 'Singapore', 'AO', 'January 24, 2026'],
    
    ## These links will need testing too
    # ['melbourne', 'Melbourne, Australia', 'AO', 'February 28, 2026'],
    # ['sydney', 'Sydney, Australia', 'AO', 'March 28, 2026'],
    # ['jakarta', 'Indonesia', 'AO', 'March 28, 2026']
]

In [ ]:

import os
import sys
# module_path = os.path.abspath(os.path.join('..'))
# if module_path not in sys.path:
#     sys.path.append(module_path)

from data_collection_pipeline import main as get_data
from data_collection_pipeline import *

print("This may take a few minutes for each event. Pleasse wait :  ^)")
for event in events:
    get_data(BASE_URL, event)
    # When finished, the data will be added to mongo db
    print('Fisnished: ', event)
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

And just like that, we have ***loads*** of data in our data base. Time to get cracking.

Or well, it would be, but I ran into a few issues with the way the data was organized,
and so we did some data wrangling:
* converted rank and wins into ints for easier analysis, and updated the pipeline
* added information about when and where the event took place into the pipeline, so that data can be analyized over time
* added everything we have so far to one giant table as well

We did some webscrabing to get the date and location of every event for every region. Perhaps next year we can make a scraper to get the event specific URLS, so we can combine these 2 tables. Toss in a feature to automatically wake up a week after each event and look for the results, and we could have the makings of a sophisticated program on our hands. Let's not get too ahead of ourselves, though...


I decided that since there are still more events to do, I'd add these steps directly into function `get_data` above, so that this can automatically be applied on the events that were marked TODO at time of writing.

# Step 3: EDA

We're back from the wragling for a second shot at analysis, and I noticed that I couldn't find 'Two Of Us, Rhyme' in any decks. I felt like that card should have been included, so let's keep that in our minds moving forward, now that we can explore every event at once. Sureley one person had to play her, right?

Hmmm... How do winrates for bosses look over time? that could be interesting.

In [4]:
EVENT = 'duluth'
HEALS = [
    "Stealth Fiend, Amaviera",
    "Incorruptible Holy Light, Eufha",
    "Lady Healer of the Creaking World",
    "Invigorate Sage",
    "Zypsophilia Fairy, Asher",
    "Two Of Us, Rhyme",
]

In [5]:
from pymongo import MongoClient

# Connect to MongoDB
username = 'sjmichael17_db_user'
password = 'rVtL43eBjseB5XkS' # plz don't hack me bro ;-;
cluster_address = 'bcsproto.peazuyx.mongodb.net/?appName=BCSproto'

client = MongoClient(f'mongodb+srv://{username}:{password}@{cluster_address}')

db = client['JSONproto']
collection = db[EVENT]

In [6]:
collection.aggregate([{$group:wins}])

SyntaxError: invalid syntax (4058584634.py, line 1)

In [ ]:


brandt
2.25
2.6
3.260869565217391
1.5
5

DE
2.652173913043478
3.0833333333333335
3.7142857142857144
4.5
4

DS
2.090909090909091
1.5
3.3157894736842106
2.5
3.1818181818181817

Lyrical bugged???

Stoi
2.1153846153846154
2.5714285714285716
2.5238095238095237
4.2
3.5

Keter
1.5
2.2857142857142856
2.875
2.6
2.5

"""
Notes:
for every nation, playing 0 counter heals lowered the win rate to below the nations average.
    Playing 2 leads to better than average results,
except for stoi, who have better than average results for player 3+
Every nation, except brandt, has the best performing count at less than 4.
    It seems 2/3 should be good, unless you're brandt gate.
    Leaning twords 3 rn, since the brandt issue may be an outlier, and the others show decent growth.
        2 may be a bit more safe, but they're both better than 0, bar brandt gate.

Lyrical didn't turn up results for Rhyme... which is cause for concearn and investigation.
The next questions I have;
    How does the precence of draws affect things?
    How does the boss interact with heal stats?
    
"""

In [4]:
EVENT = 'duluth'
HEALS = [
    "Stealth Fiend, Amaviera",
    "Incorruptible Holy Light, Eufha",
    "Lady Healer of the Creaking World",
    "Invigorate Sage",
    "Zypsophilia Fairy, Asher",
    "Two Of Us, Rhyme",
]
DRAWS = [
    'Flare Veil Dragon',
    'Rouse Wildmaster, Riley',
    'Ameliorate Connector',
    'Protection Magic, Prorobi',
    'Serene Maiden, Lena',
    'Transparent Snowy Night, Beretoi',
]